# Nom du notebook

**Définition** : Ce que je veux faire

**Formule** : 

## On commence par le chargement des données
on décommente ce dont on a besoin

In [38]:
import pandas as pd
import numpy as np

import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Si on veut lire en requêtant Elastic Search

In [30]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY = {
    "query": {
        "bool": { 
          "must": [
        
            {
              "range": {
               "logfile": {
                  "gte": "2023-01-01",
                  "lt": "2023-01-04"
               }
              }
            }

          ]
        }
      }
}

connection with ElasticSearch is KO


In [31]:
logs_january_cc = es_connector.execute_query(QUERY, "logs-new")

  0%|          | 0/361755 [00:00<?, ?it/s]

In [40]:
logs_january_cc[logs_january_cc["cc"].notna()]

,timestamp,url,idVisit,lastActionDateTime,lastActionTimestamp,referrerName,referrerTypeName,serverTimePretty,logfile,uvi,outil,outilAction,outilEvent,type,timeSpent,ccAction,query,feedbackType,visited,resultSelection,recoSelection,recoType,suggestionPrefix,suggestionSelection,cc,idCc
296,1672646193,https://code.travail.gouv.fr/contribution/quel...,29606952,2023-01-01 22:56:33,1672613793,Google,Search Engines,23:56:33,2023-01-01,-974568524,NaN,NaN,NaN,cc_select,NaN,/contribution/quelle-est-la-duree-du-preavis-e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Branche ferroviaire,NaN
736,1672645575,https://code.travail.gouv.fr/contribution/quel...,29606714,2023-01-01 22:46:15,1672613175,Google,Search Engines,23:46:15,2023-01-01,814938619,NaN,NaN,NaN,cc_select,NaN,/contribution/quelles-sont-les-conditions-dind...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Boulangerie-pâtisserie (entreprises artisanales),NaN
1222,1672613017,https://code.travail.gouv.fr/contribution/quan...,29589766,2023-01-01 13:45:21,1672580721,Google,Search Engines,14:45:21,2023-01-01,-887888452,NaN,NaN,NaN,cc_select,NaN,/contribution/quand-le-salarie-a-t-il-droit-a-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Commerce de détail et de gros à prédominance a...,NaN
1560,1672612539,https://code.travail.gouv.fr/contribution/quel...,29589615,2023-01-01 13:37:52,1672580272,Google,Search Engines,14:37:52,2023-01-01,-744520838,NaN,NaN,NaN,cc_select,NaN,/contribution/quelle-est-la-duree-du-preavis-e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Transport aérien : personnel au sol,NaN
1658,1672612491,https://code.travail.gouv.fr/contribution/le-p...,29589508,2023-01-01 13:34:51,1672580091,Google,Search Engines,14:34:51,2023-01-01,682734335,NaN,NaN,NaN,cc_select,NaN,/contribution/le-preavis-de-demission-doit-il-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pompes funèbres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359816,1672712838,https://code.travail.gouv.fr/contribution/quel...,29656346,2023-01-02 17:34:43,1672680883,,Direct Entry,18:34:43,2023-01-02,-564552324,NaN,NaN,NaN,cc_select,NaN,/contribution/quelles-sont-les-consequences-du...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sport,NaN
360353,1672713029,https://code.travail.gouv.fr/contribution/si-l...,29656491,2023-01-02 17:30:29,1672680629,Ecosia,Search Engines,18:30:29,2023-01-02,-1977008481,NaN,NaN,NaN,cc_select,NaN,/contribution/si-le-salarie-est-malade-pendant...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bureaux d'études techniques, cabinets d'ingéni...",NaN
360535,1672712951,https://code.travail.gouv.fr/contribution/quel...,29656471,2023-01-02 17:29:11,1672680551,Google,Search Engines,18:29:11,2023-01-02,2098448656,NaN,NaN,NaN,cc_select,NaN,/contribution/quelle-est-la-duree-du-preavis-e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bureaux d'études techniques, cabinets d'ingéni...",NaN
361670,1672712122,https://code.travail.gouv.fr/contribution/quan...,29654002,2023-01-02 17:20:37,1672680037,Bing,Search Engines,18:20:37,2023-01-02,-1154020765,NaN,NaN,NaN,cc_select,NaN,/contribution/quand-le-salarie-a-t-il-droit-a-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Activités industrielles de boulangerie et pâti...,NaN


In [7]:
logs_january_cc.url.value_counts()


https://code.travail.gouv.fr/convention-collective/650-metallurgie-ingenieurs-et-cadres                                                                                                            36503
https://code.travail.gouv.fr/convention-collective/1090-services-de-lautomobile-commerce-et-reparation-de-lautomobile-du-cycle                                                                     29872
https://code.travail.gouv.fr/convention-collective/573-commerces-de-gros                                                                                                                           18987
https://code.travail.gouv.fr/convention-collective/1486-bureaux-detudes-techniques-cabinets-dingenieurs-conseils-et-societes-de                                                                    17651
https://code.travail.gouv.fr/convention-collective/16-transports-routiers-et-activites-auxiliaires-du-transport                                                                                    1

In [8]:
logs_january_cc['url'] = logs_january_cc['url'].str.split('#').str[0].str.split('?').str[0]

In [11]:
logs_january_cc_visits = logs_january_cc[logs_january_cc["type"]=="visit_content"]

In [42]:
counts = logs_january_cc_visits.groupby(by="url").agg({"idVisit": "nunique"})\
    .sort_values("idVisit", ascending=False)
counts

,idVisit
url,
https://code.travail.gouv.fr/convention-collective/650-metallurgie-ingenieurs-et-cadres,13099
https://code.travail.gouv.fr/convention-collective/1090-services-de-lautomobile-commerce-et-reparation-de-lautomobile-du-cycle,10126
https://code.travail.gouv.fr/convention-collective/573-commerces-de-gros,6295
https://code.travail.gouv.fr/convention-collective/16-transports-routiers-et-activites-auxiliaires-du-transport,5278
https://code.travail.gouv.fr/convention-collective/2609-batiment-etam,4730
...,...
https://code.travail.gouv.fr/convention-collective/1518-animation).,1
https://code.travail.gouv.fr/convention-collective/86-entreprises-de-la-publicite-et-assimileescategorie%203%20niveau%202,1
https://code.travail.gouv.fr/convention-collective/86-entreprises-de-la-publicite-et-assimileescategorie 3 niveau 2,1


In [25]:
counts.to_csv("visites-cc.csv", sep=";")
